In [1]:
# mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U peft transformers datasets bitsandbytes trl==0.19.1 accelerate wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Prep dataset

In [3]:
# prepare data
from datasets import load_dataset
eval_dataset = load_dataset("FreedomIntelligence/medical-o1-verifiable-problem", split="train[:1%]")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

medical_o1_verifiable_problem.json:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40644 [00:00<?, ? examples/s]

In [4]:
eval_dataset

Dataset({
    features: ['Open-ended Verifiable Question', 'Ground-True Answer'],
    num_rows: 406
})

In [5]:
import pandas as pd
eval_dataset = pd.DataFrame(eval_dataset)
eval_dataset.head()

,Open-ended Verifiable Question,Ground-True Answer
0,An 88-year-old woman with osteoarthritis is ex...,Gastric ulcer
1,In the context of disseminated intravascular c...,Fibrin degradation products
2,"In a 3-year-old boy with severe diarrhea, vomi...","Double-stranded, icosahedral, non-enveloped"
3,Based on the chest radiograph and abdominal CT...,Hydatid Cyst
4,What is one potential side effect that is not ...,Anaphylaxis


In [6]:
# prompt_template = "You are an experienced medical professional. Answer the following medial question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: {}"

# eval_dataset['prompt'] = eval_dataset.apply(lambda x: prompt_template.format(x['Open-ended Verifiable Question']), axis=1)
# eval_dataset.prompt[0]

In [7]:
# from datasets import Dataset
# ds = Dataset.from_pandas(eval_dataset[['prompt']])
# ds

## Set up model definition

In [8]:
# with open('/content/drive/My Drive/w266/project/266 Final Project/llama3pt2-1b-peft/adapter_config.json', 'r') as f:
#   l = f.readlines()

In [9]:
# load model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.colab import userdata

# Load the Huatuo model
model_id = "FreedomIntelligence/HuatuoGPT-o1-8B"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# # Load base model
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     quantization_config=quantization_config,
#     token=userdata.get('HF_TOKEN'))

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True,
                                          token=userdata.get('HF_TOKEN'))

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [ ]:
tokenizer.get_chat_template()

'{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = "26 Jul 2024" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0][\'role\'] == \'system\' %}\n    {%- set system_message = messages[0][\'content\']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = "" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- "<|start_header_id|>system<|end_header_id|>\\n\\n" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- "Environment: ipython\\n" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- "Tools: " + builtin_tools | reject(\'equalto\', \'code_interp

## Eval of Huatuo

In [ ]:
# set up inference pipeline
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model_id,
                model_kwargs={
                    "quantization_config":quantization_config,
                },
                token=userdata.get('HF_TOKEN'))


config.json:   0%|          | 0.00/875 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
fn_out = '/content/drive/MyDrive/w266_project/HuatuoGPT-o1-8B-output-reformatted-part3.txt'
with open(fn_out, 'w') as f:
    f.write('\n')

In [ ]:
for idx,sample in enumerate(eval_dataset['Open-ended Verifiable Question']):
  if idx < 75:
    continue
  prompt = [
      {"role":"system","content":"You are a medical expert"},
      {"role":"user","content": "Provide a succinct answer to the following question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: "+sample}
  ]
  # prompt = tokenizer.apply_chat_template(prompt, tokenize=False)

  # prompt = "You are a medical expert. Provide a succinct answer to the following question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: "+sample
  for r in range(5):
    outputs = pipe(
        prompt,
        max_new_tokens=1024,
        temperature=1,
        return_full_text=False
    )
    response = outputs[0]["generated_text"]
    response = response.replace('\n','\t')
    print(response)
    with open(fn_out, 'a') as f:
        f.write(f"{idx}*****!!!!!"+response+'\n')

  if idx > 100:
    break

## Thinking		Okay, so we're trying to figure out which compounds aren't opioid agonist-antagonists from this list. Let's start with buprenorphine. Hmm, yeah, buprenorphine is known for its unique properties as an opioid partial agonist and kappa receptor antagonist. So, it fits the bill as an agonist-antagonist.		Next up is nalbuphine. This one acts like an agonist at kappa receptors but has some activity at mu receptors too. That sounds exactly like what we'd expect from an agonist-antagonist. It’s similar to buprenorphine in that sense.		Now let's look at pentazocine. Oh, yeah! Pentazocine is also well-known for having both agonist and antagonist activities at different opioid receptors. Another good example of an agonist-antagonist!		Then there's nalmefene. Wait, isn't nalmefene mainly used as a mu receptor antagonist? But wait, I think it has minimal agonistic effects. It’s primarily used in cases where you don’t want the agonist activity, more like blocking than activating.		And l

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


## Thinking		Okay, so here we have a 2-week-old baby who's got some pretty concerning symptoms. There's this blood-tinged vaginal discharge and those gray-blue patches on her skin. And she was born from a mom who had gonococcal infection during pregnancy. Hmm, that's definitely raising some red flags.		The first thing that jumps out to me is the possibility of neonatal gonorrhea. That's a common concern when a newborn shows these kinds of signs, especially given the mom's history. So, it makes sense to think about that.		Now, what do we usually do in situations like this? Well, we typically treat gonorrhea with antibiotics. But before jumping in, let’s make sure that's really what we're dealing with.		I'm thinking we need to get some tests done to confirm our suspicions. We can collect swabs from the eyes and genital area and send them off for culture. Oh, and don’t forget the Gram stain - that’s important for getting a clear picture.		Also, those gray-blue spots could actually be some